### Read in .csv files to begin removing/adding any model-specific columns

In [1]:
import pandas as pd
import numpy as np

In [2]:
flights = pd.read_csv('flights_.csv', index_col=0) # the latest flight data, with feature_engineered columns
#passengers = pd.read_csv('passengers_jan-dec_18-19.csv') # the latest passenger data

C:\Users\jsusn\anaconda3\envs\py36\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (26) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
flights.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,origin_wind_speed,origin_visibility,origin_conditions,dest_wind_speed,dest_visibility,dest_conditions,holiday,year,month,scheduled_flight_hour_of_day
60659,2018-01-01,UA,UA,UA,745,UA,N848UA,745,12953,LGA,...,14.8,9.9,Clear,12.3,9.1,Partially cloudy,True,2018,1,14
23125,2018-01-01,UA,UA_CODESHARE,UA,3582,YX,N649RW,3582,11298,DFW,...,15.0,9.9,Partially cloudy,12.3,9.1,Partially cloudy,True,2018,1,20
3646,2018-01-01,UA,UA,UA,1591,UA,N87527,1591,11298,DFW,...,15.0,9.9,Partially cloudy,5.6,4.2,Clear,True,2018,1,5
73558,2018-01-01,WN,WN,WN,5500,WN,N8685B,5500,11292,DEN,...,12.3,9.1,Partially cloudy,14.6,9.9,Partially cloudy,True,2018,1,18
77601,2018-01-01,WN,WN,WN,415,WN,N8690A,415,12892,LAX,...,5.6,4.2,Clear,12.7,9.9,Clear,True,2018,1,12


### One-Hot encoding for:
- weather
- origin_city_name and dest_city_name
- day_of_week

In [4]:
flights.columns

Index(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time', 'dep_time',
       'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in',
       'crs_arr_time', 'arr_time', 'arr_delay', 'cancelled',
       'cancellation_code', 'diverted', 'crs_elapsed_time',
       'actual_elapsed_time', 'air_time', 'distance', 'carrier_delay',
       'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay',
       'first_dep_time', 'total_add_gtime', 'longest_add_gtime',
       'day_of_the_week', 'origin_wind_speed', 'origin_visibility',
       'origin_conditions', 'dest_wind_speed', 'dest_visibility',
       'dest_conditions', 'holiday', 'year', 'month',
       'scheduled_flight_hour_of_day'],
      dtype='object')

In [5]:
dest_conditions = pd.get_dummies(flights['dest_conditions'], prefix='dest')
origin_conditions = pd.get_dummies(flights['origin_conditions'], prefix='origin')

origin_city_name = pd.get_dummies(flights['origin_city_name'], prefix='origin')
dest_city_name = pd.get_dummies(flights['dest_city_name'], prefix='dest')

day_of_week = pd.get_dummies(flights['day_of_the_week'])

In [6]:
df_flights = pd.concat((flights['year'], flights['month'], day_of_week, origin_city_name, origin_conditions, flights['origin_visibility'], flights['origin_wind_speed'], dest_city_name, dest_conditions, flights['dest_visibility'], flights['dest_wind_speed'], flights['holiday'], flights['arr_delay']), axis=1)

In [7]:
df_flights = df_flights.dropna()

# Linear Regression
### Model-Specific

In [76]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.utils import resample,shuffle

from sklearn import metrics
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import r2_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score

import xgboost as xgb
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor

In [9]:
X = df_flights.drop(columns=['arr_delay'])
y = df_flights['arr_delay']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=101)

### Baseline model

In [11]:
clf = LinearRegression()
clf.fit(X_train, y_train)

LinearRegression()

In [12]:
y_preds = clf.predict(X_test)

In [13]:
clf.score(X_test, y_test)

0.03995540852806634

In [14]:
# can only go up from here

In [72]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear')

In [73]:
xg_reg.fit(X_train, y_train)

[17:27:02] WARNING: d:\bld\xgboost-split_1631904903843\work\src\objective\regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=16, num_parallel_tree=1,
             objective='reg:linear', random_state=0, reg_alpha=0, reg_lambda=1,
             scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [74]:
y_preds = xg_reg.predict(X_test)

In [75]:
xg_reg.score(X_test, y_test)

0.13055764093736877

In [19]:
rmse = np.sqrt(mean_squared_error(y_test, y_preds))
print("RMSE: %f" % (rmse))

RMSE: 48.098899


### Some tweaking

In [103]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', learning_rate = 0.2, max_depth = 7, alpha = 0.1, n_estimators = 100)

In [104]:
xg_reg.fit(X_train, y_train)

[17:31:10] WARNING: d:\bld\xgboost-split_1631904903843\work\src\objective\regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(alpha=0.1, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.2, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=16, num_parallel_tree=1,
             objective='reg:linear', random_state=0, reg_alpha=0.100000001,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [105]:
y_preds = xg_reg.predict(X_test)

In [106]:
xg_reg.score(X_test, y_test)

0.13267078544586453

In [102]:
rmse = np.sqrt(mean_squared_error(y_test, y_preds))
print("RMSE: %f" % (rmse))

RMSE: 45.558996


### More tweaking...

In [ ]:
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

In [107]:
# https://towardsdatascience.com/doing-xgboost-hyper-parameter-tuning-the-smart-way-part-1-of-2-f6d255a45dde
# https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/

In [ ]:
# grid = GridSearchCV(estimator=xgb, param_grid=params, scoring='score', n_jobs=1, cv=skf.split(X,Y), verbose=3 )
# grid.fit(X, Y)
# print('\n All results:')
# print(grid.cv_results_)
# print('\n Best estimator:')
# print(grid.best_estimator_)
# print('\n Best score:')
# print(grid.best_score_ * 2 - 1)
# print('\n Best parameters:')
# print(grid.best_params_)
# results = pd.DataFrame(grid.cv_results_)
# results.to_csv('xgb-grid-search-results-01.csv', index=False)

# y_test = grid.best_estimator_.predict_proba(test)
# results_df = pd.DataFrame(data={'id':test_df['id'], 'target':y_test[:,1]})
# results_df.to_csv('submission-grid-search-xgb-porto-01.csv', index=False)

In [25]:
#xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.01, max_depth = 5, alpha = 1, n_estimators = 10000)

In [26]:
xg_reg.fit(X_train, y_train)

XGBRegressor(alpha=1, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.01, max_delta_step=0, max_depth=5,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=10000, n_jobs=16, num_parallel_tree=1, random_state=0,
             reg_alpha=1, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [27]:
y_preds = xg_reg.predict(X_test)

In [28]:
xg_reg.score(X_test, y_test)

0.13612371900916242

In [29]:
rmse = np.sqrt(mean_squared_error(y_test, y_preds))
print("RMSE: %f" % (rmse))

RMSE: 45.444143
